In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce

from sklearn.model_selection import train_test_split

from sklearn import tree

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.utils.extmath import density

from sklearn import metrics
import model
from model import Model
import health_data
from health_data import DepressionHealthData
import imp


#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)


In [14]:
dhd = DepressionHealthData()
dhd.prepare_data()
dhd.df.reset_index(drop=True, inplace=True)

Read Health Data
    SEQN  ACD011A  ACD011B  ACD011C  ACD040  ACD110  ALQ101  ALQ110  ALQ120Q  \
0  73557      1.0      NaN      NaN     NaN     NaN     1.0     NaN      1.0   
1  73558      1.0      NaN      NaN     NaN     NaN     1.0     NaN      7.0   
2  73559      1.0      NaN      NaN     NaN     NaN     1.0     NaN      0.0   
3  73560      1.0      NaN      NaN     NaN     NaN     NaN     NaN      NaN   
4  73561      1.0      NaN      NaN     NaN     NaN     1.0     NaN      0.0   

   ALQ120U  ALQ130  ALQ141Q  ALQ141U  ALQ151  ALQ160  BPQ020  BPQ030  BPD035  \
0      3.0     1.0      0.0      NaN     1.0     NaN     1.0     1.0    62.0   
1      1.0     4.0      2.0      1.0     1.0     0.0     1.0     1.0    53.0   
2      NaN     NaN      NaN      NaN     2.0     NaN     1.0     1.0    40.0   
3      NaN     NaN      NaN      NaN     NaN     NaN     NaN     NaN     NaN   
4      NaN     NaN      NaN      NaN     2.0     NaN     1.0     2.0    55.0   

   BPQ040A  BPQ050A  

In [15]:
sum = dhd.get_depression_status(classify=True)
# sum = dhd.get_depression_status(classify=False)
# dhd.clean_data()
df = dhd.df
# df.head()

Function: get_depression_status Total Time Used: 0.13650798797607422


In [16]:
# def range_encoding(df: DataFrame, encode_name: str, start: float, end: float, step: float, special: dict):
#   for interval in range(start, end, step):
#     df.insert(df.columns.get_loc(encode_name), f"{encode_name}_{interval}_{interval + step}", df[encode_name].apply(lambda x: 1 if x >= interval and x < interval + step else 0))
#   for sp in special:
#     df.insert(df.columns.get_loc(encode_name), f"{encode_name}_{sp}", df[encode_name].apply(lambda x: 1 if str(x).lower() == str(sp).lower() or (str(x).lower() != str(np.NaN).lower() and str(sp).lower() != str(np.NaN).lower() and str(int(x)) == str(int(sp))) else 0))
#   df.drop([encode_name], inplace=True, axis=1)

def range_encoding(df: DataFrame, encode_name: str, start: float, end: float, step: float, special: list):
  # for interval in range(start, end, step):
  #   df.insert(df.columns.get_loc(encode_name), f"{encode_name}_{interval}_{interval + step}", df[encode_name].apply(lambda x: 1 if x >= interval and x < interval + step else 0))
  for sp in special:
    df.insert(df.columns.get_loc(encode_name), f"{encode_name}_{sp}", df[encode_name].apply(lambda x: 1 if str(x).lower() == str(sp).lower() or (str(x).lower() != str(np.NaN).lower() and str(sp).lower() != str(np.NaN).lower() and str(int(x)) == str(int(sp))) else 0))
  df.insert(df.columns.get_loc(encode_name), f"{encode_name}_RE", df[encode_name].apply(lambda x: 0 if any(str(x).lower() in str(sp).lower() or (not str(x).lower() in str(np.NaN).lower() and not str(sp).lower() in str(np.NaN).lower() and str(int(x)) in str(int(sp))) for sp in special) else x))
  df.drop([encode_name], inplace=True, axis=1)


In [17]:
top25 = ["RXDCOUNT", "PFQ063B", "SLQ050", "HSD010", "SLD010H", "WTINT2YR", "WHQ150", "WHD010", "PAQ710", "INDFMMPI", "CDQ010", "MDHRAGE", "INDFMPIR", "CBD070", "CBD110", "SMD030", "SXD031", "IND235", "CBD120", "ALQ130"]
# top25onehot = ["PFQ063B", "SLQ050", "HSD010", "SXQ709", "IND235", "ALQ101", "DMDEDUC2", "DMDEDUC3", "DLQ050", "MCQ300C", "HUQ051", "CBQ580", "SMQ040", "HUQ010", "MCQ092", "PFQ063A", "PFQ063D"]
top25onehot = ["PFQ063B", "SLQ050", "HSD010", "SXQ709", "IND235", "ALQ101", "DLQ050", "MCQ300C", "HUQ051", "CBQ580", "SMQ040", "HUQ010", "MCQ092", "PFQ063A", "PFQ063D"]
NaNDict = ["NaN"]
RefusedIDK1 = ["7", "9"]
RefusedIDK2 = ["77", "99"]
RefusedIDK3 = ["777", "999"]
RefusedIDK4 = ["7777", "9999"]
RefusedIDK5 = ["77777", "99999"]
RefusedIDK6 = ["777777", "999999"]
RXDCOUNT = ["RXDCOUNT", 1, 23, 2, NaNDict]
SLD010H = ["SLD010H", 2, 12, 2, [*NaNDict, *RefusedIDK2]]
WHD010 = ["WHD010", 48, 81, 5, [*NaNDict, *RefusedIDK4]]
df[df["PAQ710"] == 8] = 0
PAQ710 = ["PAQ710", 0, 5, 1, [*NaNDict, *RefusedIDK2]]
INDFMMPI = ["INDFMMPI", 0, 5, 1, [*NaNDict]]
CBD110 = ["CBD110", 0, 2000, 200, [*NaNDict, *RefusedIDK6]]
SMD030 = ["SMD030", 0, 64, 10, [*NaNDict, *RefusedIDK3, "80"]]
SXD031 = ["SXD031", 9, 60, 5, [*NaNDict, *RefusedIDK2]]
CBD070 = ["CBD070", 0, 4285, 500, [*NaNDict, *RefusedIDK6]]
CBD090 = ["CBD090", 0, 1542, 200, [*NaNDict, *RefusedIDK6]]
ALQ130 = ["ALQ130", 1, 25, 5, [*NaNDict, *RefusedIDK3]]
ALQ120Q = ["ALQ120Q", 0, 365, 30, [*NaNDict, *RefusedIDK3]]
WHD050 = ["WHD050", 75, 559, 20, [*NaNDict, *RefusedIDK4]]
RHD190 = ["RHD190", 14, 45, 5, [*NaNDict, *RefusedIDK3]]
RIDAGEYR = ["RIDAGEYR", 0, 80, 5, [*NaNDict]]
HOD050 = ["HOD050", 1, 13, 2, [*NaNDict, *RefusedIDK3]]
WHD020 = ["WHD020", 75, 493, 20, [*NaNDict, *RefusedIDK4]]
DBD895 = ["DBD895", 1, 27, 3, [*NaNDict, *RefusedIDK4, "5555"]]
OHQ870 = ["OHQ870", 0, 7, 2, [*NaNDict, *RefusedIDK2]]


# top25_range_encoding = [RXDCOUNT, SLD010H, WHD010, PAQ710, INDFMMPI, CBD110, SMD030, SXD031, CBD070, CBD090, ALQ130, ALQ120Q, WHD050, RIDAGEYR, RHD190, HOD050, WHD020, DBD895, OHQ870]
top25_range_encoding = [SLD010H, WHD010, PAQ710, INDFMMPI, CBD110, SMD030, SXD031, CBD070, CBD090, ALQ130, ALQ120Q, WHD050, RHD190, HOD050, WHD020, DBD895, OHQ870]
top25_range_encoding_columns = [*[col[0] for col in top25_range_encoding], *top25onehot]
df_range_encoding = df[top25_range_encoding_columns].copy()
df_range_encoding = pd.get_dummies(df_range_encoding, columns=top25onehot)
for re in top25_range_encoding:
  print(f"encoding {re[0]}, {re[1]}, {re[2]}, {re[3]}, {re[4]}")
  range_encoding(df_range_encoding, re[0], re[1], re[2], re[3], re[4])

df_range_encoding.insert(0, 'SUM', sum)
# df_range_encoding = df.drop(['SEQN','DPQ010', 'DPQ020',	'DPQ030',	'DPQ040',	'DPQ050',	'DPQ060',	'DPQ070',	'DPQ080',	'DPQ090'], axis=1)
# df_range_encoding.fillna(0, inplace=True)
# df_range_encoding = pd.get_dummies(df_range_encoding, columns=df_range_encoding.columns[1:])

/var/folders/m7/7mcwkh2x2rx7v7p1s3m7pqlw0000gn/T/ipykernel_44634/1192084405.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["PAQ710"] == 8] = 0
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


encoding SLD010H, 2, 12, 2, ['NaN', '77', '99']
encoding WHD010, 48, 81, 5, ['NaN', '7777', '9999']
encoding PAQ710, 0, 5, 1, ['NaN', '77', '99']
encoding INDFMMPI, 0, 5, 1, ['NaN']
encoding CBD110, 0, 2000, 200, ['NaN', '777777', '999999']
encoding SMD030, 0, 64, 10, ['NaN', '777', '999', '80']
encoding SXD031, 9, 60, 5, ['NaN', '77', '99']
encoding CBD070, 0, 4285, 500, ['NaN', '777777', '999999']
encoding CBD090, 0, 1542, 200, ['NaN', '777777', '999999']
encoding ALQ130, 1, 25, 5, ['NaN', '777', '999']
encoding ALQ120Q, 0, 365, 30, ['NaN', '777', '999']
encoding WHD050, 75, 559, 20, ['NaN', '7777', '9999']
encoding RHD190, 14, 45, 5, ['NaN', '777', '999']
encoding HOD050, 1, 13, 2, ['NaN', '777', '999']
encoding WHD020, 75, 493, 20, ['NaN', '7777', '9999']
encoding DBD895, 1, 27, 3, ['NaN', '7777', '9999', '5555']
encoding OHQ870, 0, 7, 2, ['NaN', '77', '99']


In [3]:
 #获取分类变量的名称
 15     cat_cols = [x for x in category_data.columns]
 16     
 17     #以字典形式存储分类变量的{名称：维度}
 18     category_origin_dimension = [math.ceil(category_data[cat_col].drop_duplicates().size) for cat_col in cat_cols]
 19     
 20     #以字典形式存储分类变量 计划 在embed后的{名称：维度}
 21     category_embedding_dimension = [math.ceil(math.sqrt(category_data[cat_col].drop_duplicates().size)) for cat_col in cat_cols]

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import Adam
import torch.nn.functional as F
from tqdm import tqdm

dtype = torch.FloatTensor

# sentences = ["i like dog", "i love coffee", "i hate milk", "i do nlp"]

# word_list = ' '.join(sentences).split()
# word_list = list(set(word_list))

# word_dict = {w: i for i, w in enumerate(word_list)}
# number_dict = {i: w for i, w in enumerate(word_list)}
# print(word_dict)


In [8]:
n_class = len(df_range_encoding.columns)

m = 5
n_step = 2
n_hidden = 2

# df_range_encoding_input = pd.merge(df_range_encoding.iloc[:,0:1], df_range_encoding.iloc[:,2:], left_index=True, right_index=True)
# df_range_encoding_output = df_range_encoding.iloc[:, 1]
#df_range_encoding.insert(0, 'SUM', sum)

df_range_encoding_input = df_range_encoding.iloc[:,1:]
df_range_encoding_output = df_range_encoding.iloc[:, 0]

# df_range_encoding_input = df.iloc[:,1:]
# df_range_encoding_output = df.iloc[:, 0]

#pca = PCA(n_components=10)
#pca.fit(df_range_encoding_input)
#df_range_encoding_input = pca.transform(df_range_encoding_input)

df_range_encoding_output.head()

0    0.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: SUM, dtype: float64

In [10]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class HealthData(Dataset):
  def __init__(self, input_df: DataFrame, output_df: DataFrame, mode = 'train', enable_pca = False):
    self.mode = mode
    '''
    <------Some Data Preprocessing---------->
    Removing Null Values, Outliers and Encoding the categorical labels etc
    '''
    if enable_pca:
      self.inp = input_df
      self.oup = output_df.values.reshape(input_df.shape[0],1)
    elif self.mode == 'train':
      self.inp = input_df.values
      self.oup = output_df.values.reshape(input_df.shape[0],1)
    else:
      self.inp = df.values
 
  def __len__(self):
    return len(self.inp)
  
  def __getitem__(self, idx):
    if self.mode == 'train':
      inpt  = torch.Tensor(self.inp[idx])
      oupt  = torch.Tensor(self.oup[idx])
      return inpt, oupt
    else:
      inpt = torch.Tensor(self.inp[idx])
      return inpt

In [11]:

def swish(x):
    return x * torch.sigmoid(x)

class Network(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(199, 16)
        self.b1 = nn.BatchNorm1d(16)
        self.fc2 = nn.Linear(16, 8)
        self.b2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8,4)
        self.b3 = nn.BatchNorm1d(4)
        self.fc4 = nn.Linear(4,1)

    def forward(self,x):
        x = swish(self.fc1(x))
        x = self.b1(x)
        x = swish(self.fc2(x))
        x = self.b2(x)
        x = swish(self.fc3(x))
        x = self.b3(x)
        x = torch.sigmoid(self.fc4(x))
        return x

# class Network(nn.Module):

#     def __init__(self):
#         super().__init__()
#         self.fc1 = nn.Linear(10, 4)
#         self.b1 = nn.BatchNorm1d(4)
#         self.fc4 = nn.Linear(4,1)

#     def forward(self,x):
#         x = swish(self.fc1(x))
#         x = self.b1(x)
#         x = swish(self.fc4(x))
#         return x

def train(model, x, y, optimizer, criterion):
    model.zero_grad()
    #print('x      size = ', x.size())
    output = model(x)
    #print('output size = ', output.size())
    #print('y      size = ', y.size())
    #print(f"x: {x.shape}, y: {y.squeeze().long().unique()}, output: {output.shape}")
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()
    return loss, output

def plot_normal_loss(train_score, loss, train_size):
    train_scores_mean = np.mean(train_score, axis=1)
    train_scores_std = np.std(train_score, axis=1)
    loss_mean = np.mean(loss, axis=1)
    loss_std = np.std(loss, axis=1)

    #print(f"ts: {train_score.shape} loss: {loss.shape} tsize: {train_size.shape}")

    plt.fill_between(train_size, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1,
                        color="r")
    plt.fill_between(train_size, loss_mean - loss_std,
                        loss_mean + loss_std, alpha=0.1, color="g")

    plt.plot(train_size, train_scores_mean, 'o--', color="r",
                label="Accuracy Score")
    plt.plot(train_size, loss_mean, 'o-', color="g",
                label="Loss")

    plt.grid()
    plt.title(f'Learn Curve')
    plt.legend(loc="best")
    plt.show()



def plot_cv(train_score, test_score, train_size):
    train_scores_mean = np.mean(train_score, axis=1)
    train_scores_std = np.std(train_score, axis=1)
    test_scores_mean = np.mean(test_score, axis=1)
    test_scores_std = np.std(test_score, axis=1)

    print(f"trs: {train_score.shape} tes: {test_score.shape} tsize: {train_size.shape}")

    plt.fill_between(train_size, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1,
                        color="r")
    plt.fill_between(train_size, test_scores_mean - test_scores_std,
                        test_scores_mean + test_scores_std, alpha=0.1, color="g")

    plt.plot(train_size, train_scores_mean, 'o--', color="r",
                label="Training score")
    plt.plot(train_size, test_scores_mean, 'o-', color="g",
                label="Cross-validation score")

    plt.grid()
    plt.title(f'Cross Validation Curve')
    plt.legend(loc="best")
    plt.show()
    print(f"Train Score: {train_score.mean()} Validation Score: {test_score.mean()}")


from sklearn.model_selection import KFold
from sklearn import metrics

fold_count = 4
folder = KFold(n_splits=fold_count)

train_score = []
test_score = []

for (trn_idx, val_idx) in folder.split(df_range_encoding_input, df_range_encoding_output):
    train_, val = df_range_encoding.iloc[trn_idx], df_range_encoding.iloc[val_idx]
    train_X = train_.iloc[:, 1:]
    train_y = train_.iloc[:, 0]
    val_X = val.iloc[:, 1:]
    val_y = val.iloc[:, 0]
    BATCH_SIZE = 64
    ## Initialize the DataSet
    data = HealthData(train_X, train_y)
    ## Load the Dataset
    data_train = DataLoader(dataset = data, batch_size = BATCH_SIZE, shuffle = True)
    ## device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = Network()
    EPOCHS = 100
    optm = Adam(net.parameters(), lr = 0.001)
    # criterion = nn.MSELoss()
    criterion = nn.BCELoss()
    
    loss_scores = []
    accuracy_scores = []

    train_subscore = []
    test_subscore = []

    for epoch in range(EPOCHS):
        epoch_loss = 0
        correct = 0
        loss_c = 0
        loss_subscores = []
        acc_subscores = []
        for bidx, (inp, oup) in tqdm(enumerate(data_train)):
            x_train, y_train = inp, oup
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            loss, predictions = train(net,x_train,y_train, optm, criterion)
            for idx, i in enumerate(predictions):
                i  = torch.round(i)
                #print(i)
                if i == y_train[idx]:
                    correct += 1
            acc = (correct/len(data))
            loss_subscores.append(loss.detach().numpy())
            acc_subscores.append(acc)
            epoch_loss+=loss
            loss_c += 1
            #print('Loss : {}'.format(loss))
        loss_scores.append(loss_subscores)
        accuracy_scores.append(acc_subscores)
        print('Epoch {} Accuracy : {}'.format(epoch+1, acc*100))
        print('Epoch {} Loss : {}'.format((epoch+1),epoch_loss/loss_c))
        train_subscore.append(metrics.accuracy_score(train_y,net(torch.Tensor(train_X.values)).round().detach().numpy()))
        test_subscore.append(metrics.accuracy_score(val_y,net(torch.Tensor(val_X.values)).round().detach().numpy()))
        # train_subscore.append(metrics.mean_squared_error(train_y,net(torch.Tensor(train_X)).detach().numpy()))
        # test_subscore.append(metrics.mean_squared_error(val_y,net(torch.Tensor(val_X)).detach().numpy()))
    
    
    val_y_hat = net(torch.Tensor(val_X.values)).detach().numpy()
    # val_y_hat = net(torch.Tensor(val_X)).detach().numpy()
    print(f"Score - {metrics.mean_squared_error(val_y,val_y_hat)}")
    train_score.append(train_subscore)
    test_score.append(test_subscore)
    break

pd.DataFrame({"y": val_y.tolist(), "y_hat": val_y_hat.tolist()})
    #plot_normal_loss(np.array(accuracy_scores), np.array(loss_scores), np.linspace(0, len(train_), epoch+1))

#plot_cv(np.array(train_score), np.array(test_score), np.linspace(0, len(train_)*fold_count, fold_count))

# class NNLM(nn.Module):
#     def __init__(self):
#         super(NNLM, self).__init__()
#         self.embed = nn.Embedding(n_class, m)
#         self.W = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype))
#         self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))
#         self.U = nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))
#         self.b = nn.Parameter(torch.randn(n_class).type(dtype))

#     def forward(self, x):
#         x = self.embed(x)  # 4 x 2 x 2
#         x = x.view(-1, n_step * m)
#         tanh = torch.tanh(self.d + torch.mm(x, self.W))  # 4 x 2
#         output = self.b + torch.mm(tanh, self.U)
#         return output


# model = NNLM()

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# input_batch, target_batch = make_batch(sentences)
# input_batch = Variable(torch.LongTensor(input_batch))
# target_batch = Variable(torch.LongTensor(target_batch))

# for epoch in range(int(1e4)):
#     optimizer.zero_grad()

#     output = model(input_batch)  # input: 4 x 2

#     loss = criterion(output, target_batch)

#     if (epoch + 1) % 1000 == 0:
#         print('epoch:', '%04d' % (epoch + 1), 'cost = {:.6f}'.format(loss.item()))

#     loss.backward()
#     optimizer.step()

# predict = model(input_batch).data.max(1, keepdim=True)[1]

# print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])






64it [00:00, 219.12it/s]


Epoch 1 Accuracy : 81.28553770086526
Epoch 1 Loss : 0.555204451084137


64it [00:00, 253.05it/s]


Epoch 2 Accuracy : 86.99629171817058
Epoch 2 Loss : 0.5006985068321228


64it [00:00, 259.39it/s]


...

64it [00:00, 252.79it/s]


Epoch 99 Accuracy : 91.19901112484548
Epoch 99 Loss : 0.2421683818101883


64it [00:00, 252.06it/s]


Epoch 100 Accuracy : 91.02595797280594
Epoch 100 Loss : 0.23905286192893982
Score - 0.08873769683816549


,y,y_hat
0,0.0,[0.03452875465154648]
1,0.0,[0.2090614289045334]
2,0.0,[0.008891595527529716]
3,1.0,[0.10527012497186661]
4,1.0,[0.17396752536296844]
5,0.0,[0.0007986287819221616]
6,0.0,[0.1992841511964798]
7,0.0,[0.04106547310948372]
8,0.0,[0.044793907552957535]
9,0.0,[0.008658058010041714]


In [205]:
df.shape

(5394, 954)

In [12]:
import imp
imp.reload(model)
imp.reload(health_data)
from model import Model
